In [1]:
import numpy as np
from enum import Enum, auto

In [2]:
class SquareType(Enum):
    START = auto()
    GOAL = auto()
    CLIFF = auto()
    THROUGH = auto()


class Scene:
    def __init__(self):
        self._start_pos = (-1, 0)
        self._goal_pos = (-1, -1)
        self._scene = self.define_scene(4, 12, start_pos=self._start_pos, goal_pos=self._goal_pos)
        self.square_rep = {
            SquareType.CLIFF: 'C',
            SquareType.START: 'S',
            SquareType.GOAL: 'G',
            SquareType.THROUGH: ' '
        }

    @property
    def scene(self):
        return self._scene

    @property
    def start_pos(self):
        return self._start_pos

    @property
    def goal_pos(self):
        return self._goal_pos

    @staticmethod
    def define_scene(depth: int, width: int, start_pos: tuple[int, int], goal_pos: tuple[int, int]):
        scene = np.full((depth, width), fill_value=SquareType.THROUGH)
        scene[-1, :] = SquareType.CLIFF
        scene[*start_pos] = SquareType.START
        scene[*goal_pos] = SquareType.GOAL

        return scene

    def __str__(self):

        rows = []
        separator_row = "\n" + "|-----" * self._scene.shape[1] + "|\n"
        for row in self._scene:
            rows.append('|' +  '|'.join(f"  {self.square_rep[v]}  " for v in row) + '|')

        ss = separator_row.join(rows)
        ss = separator_row + ss + separator_row
        return ss

s = Scene()
print(s)



|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|     |     |     |     |     |     |     |     |     |     |     |     |
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|     |     |     |     |     |     |     |     |     |     |     |     |
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|     |     |     |     |     |     |     |     |     |     |     |     |
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|  S  |  C  |  C  |  C  |  C  |  C  |  C  |  C  |  C  |  C  |  C  |  G  |
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|

